## Notebook: Transform dữ liệu từ tầng broze lên tầng silver

### 1: Import & Connect DB

In [ ]:
import os
import json
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import BigInteger, Integer, Float, DateTime, Text, VARCHAR
from dotenv import load_dotenv

# Load biến môi trường
load_dotenv()

DB_USER   = os.getenv("DB_USER")
DB_PASS   = os.getenv("DB_PASS")
DB_HOST   = os.getenv("DB_HOST")
DB_PORT   = os.getenv("DB_PORT")
DB_BRONZE = os.getenv("DB_BRONZE")
DB_SILVER = os.getenv("DB_SILVER")

# Tạo kết nối
bronze_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BRONZE}")
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")

print("✅ Đã kết nối Bronze và Silver DB")


✅ Đã kết nối Bronze và Silver DB


### 2. Load dữ liệu từ Bronze

In [42]:
orders_df = pd.read_sql("SELECT * FROM orders_raw", bronze_engine)
print("Shape:", orders_df.shape)

# Hiển thị 1 record mẫu để nắm cấu trúc
print(json.dumps(json.loads(orders_df["raw_json"].iloc[169]), indent=2, ensure_ascii=False))


Shape: (40236, 4)
{
  "total_quantity": 2,
  "transfer_money": 0,
  "charged_by_momo": 0,
  "shop_id": 230361475,
  "prepaid": 0,
  "inserted_at": "2025-04-14T23:51:24.740907",
  "order_sources_name": "Facebook",
  "is_live_shopping": false,
  "pke_mkter": null,
  "note_image": null,
  "status": 3,
  "ads_source": "Facebook",
  "p_utm_id": null,
  "order_link": "https://pos.pages.fm/shop/230361475/order?order_id=270226784656452",
  "items_length": 2,
  "total_price": 758000,
  "tags": [
    {
      "id": 4,
      "name": "Chưa đối soát"
    }
  ],
  "conversation_id": "461234203738139_29023387273972369",
  "assigning_care": null,
  "post_id": "461234203738139_122119738148572601",
  "p_utm_campaign": null,
  "advanced_platform_fee": {},
  "note_print": null,
  "assigning_care_id": null,
  "sub_status": null,
  "p_utm_content": null,
  "bill_email": null,
  "is_livestream": false,
  "total_price_after_sub_discount": 700000,
  "customer_needs": [],
  "marketplace_id": null,
  "total_disco

### 3: Liệt kê toàn bộ keys trong JSON

In [ ]:

all_keys = set()
for raw in orders_df["raw_json"].head(200):   # scan 200 record đầu
    d = json.loads(raw)
    all_keys.update(d.keys())

print("===== Tổng số keys:", len(all_keys), " =====")
print(sorted(all_keys))


===== Tổng số keys: 98  =====
['account', 'account_name', 'activated_combo_products', 'activated_promotion_advances', 'ad_id', 'ads_source', 'advanced_platform_fee', 'assigning_care', 'assigning_care_id', 'assigning_seller', 'assigning_seller_id', 'bank_payments', 'bill_email', 'bill_full_name', 'bill_phone_number', 'botcake_info', 'buyer_total_amount', 'cash', 'charged_by_card', 'charged_by_momo', 'charged_by_qrpay', 'cod', 'conversation_id', 'creator', 'customer', 'customer_needs', 'customer_pay_fee', 'customer_referral_code', 'estimate_delivery_date', 'exchange_payment', 'exchange_value', 'fee_marketplace', 'histories', 'id', 'inserted_at', 'is_exchange_order', 'is_free_shipping', 'is_live_shopping', 'is_livestream', 'is_smc', 'items', 'items_length', 'last_editor', 'levera_point', 'link', 'link_confirm_order', 'marketer', 'marketplace_id', 'money_to_collect', 'note', 'note_image', 'note_print', 'order_currency', 'order_link', 'order_sources', 'order_sources_name', 'p_utm_campaign',

### 4. Parse JSON thành Fact & Dim

In [ ]:
# ---- Fact Orders ----
def parse_order(row):
    d = json.loads(row["raw_json"])
    return {
        "order_id": d.get("id"),
        "shop_id": d.get("shop_id"),
        "page_id": d.get("page_id"),
        "customer_id": d.get("customer", {}).get("id"),
        "warehouse_id": d.get("warehouse_id"),

        "status": d.get("status"),
        "status_name": d.get("status_name"),
        "sub_status": d.get("sub_status"),

        "inserted_at": d.get("inserted_at"),
        "updated_at": d.get("updated_at"),
        "time_assign_care": d.get("time_assign_care"),
        "time_assign_seller": d.get("time_assign_seller"),
        "estimate_delivery_date": d.get("estimate_delivery_date"),

        "total_quantity": d.get("total_quantity"),
        "total_price": d.get("total_price"),
        "total_discount": d.get("total_discount"),
        "total_price_after_sub_discount": d.get("total_price_after_sub_discount"),

        "cod": d.get("cod"),
        "cash": d.get("cash"),
        "transfer_money": d.get("transfer_money"),
        "charged_by_momo": d.get("charged_by_momo"),
        "charged_by_card": d.get("charged_by_card"),
        "charged_by_qrpay": d.get("charged_by_qrpay"),
        "exchange_payment": d.get("exchange_payment"),
        "prepaid": d.get("prepaid"),

        "money_to_collect": d.get("money_to_collect"),
        "shipping_fee": d.get("shipping_fee"),
        "partner_fee": d.get("partner_fee"),
        "surcharge": d.get("surcharge"),
        "fee_marketplace": d.get("fee_marketplace"),
        "tax": d.get("tax")
    }

orders_parsed = orders_df.apply(parse_order, axis=1, result_type="expand")
print("✅ fact_orders shape:", orders_parsed.shape)


# ---- Fact Order Items ----
def parse_order_items(row):
    d = json.loads(row["raw_json"])
    order_id = d.get("id")

    items = d.get("items", [])
    results = []
    for item in items:
        results.append({
            "order_id": order_id,
            "product_id": item.get("product_id"),
            "variation_id": item.get("variation_id"),
            "sku": item.get("sku"),
            "name": item.get("name"),
            "quantity": item.get("quantity"),
            "price": item.get("price"),
            "discount": item.get("discount"),
            "total_price": item.get("total_price"),
            "note": item.get("note")
        })
    return results

order_items_records = []
for _, row in orders_df.iterrows():
    order_items_records.extend(parse_order_items(row))

order_items_parsed = pd.DataFrame(order_items_records)
print("✅ fact_order_items shape:", order_items_parsed.shape)


# ---- Dim Pages ----
def parse_page(row):
    d = json.loads(row["raw_json"])
    page = d.get("page", {})
    return {
        "page_id": page.get("id"),
        "page_name": page.get("name"),
        "page_username": page.get("username"),
        "account": d.get("account"),
        "account_name": d.get("account_name")
    }

pages_parsed = orders_df.apply(parse_page, axis=1, result_type="expand").drop_duplicates("page_id")
print("✅ dim_pages shape:", pages_parsed.shape)


# ---- Dim Warehouses ----
def parse_warehouse(row):
    d = json.loads(row["raw_json"])
    wh = d.get("warehouse_info", {})
    return {
        "warehouse_id": d.get("warehouse_id"),
        "warehouse_name": wh.get("name"),
        "warehouse_address": wh.get("full_address"),
        "province_id": wh.get("province_id"),
        "district_id": wh.get("district_id"),
        "commune_id": wh.get("commune_id"),
        "phone_number": wh.get("phone_number")
    }

warehouses_parsed = orders_df.apply(parse_warehouse, axis=1, result_type="expand").drop_duplicates("warehouse_id")
print("✅ dim_warehouses shape:", warehouses_parsed.shape)


# ---- Dim Shipping Addresses ----
def parse_shipping_address(row):
    d = json.loads(row["raw_json"])
    sa = d.get("shipping_address", {})
    return {
        "order_id": d.get("id"),
        "full_name": sa.get("full_name"),
        "phone_number": sa.get("phone_number"),
        "full_address": sa.get("full_address"),
        "province_id": sa.get("province_id"),
        "province_name": sa.get("province_name"),
        "district_id": sa.get("district_id"),
        "district_name": sa.get("district_name"),
        "commune_id": sa.get("commune_id"),
        "commune_name": sa.get("commune_name")
    }

shipping_addresses_parsed = orders_df.apply(parse_shipping_address, axis=1, result_type="expand")
print("✅ dim_shipping_addresses shape:", shipping_addresses_parsed.shape)


✅ fact_orders shape: (40236, 31)
✅ fact_order_items shape: (46611, 10)
✅ dim_pages shape: (74, 5)
✅ dim_warehouses shape: (24, 7)
✅ dim_shipping_addresses shape: (40236, 10)


### 5. Data Quality Check

In [ ]:


def data_quality_report(df, pk=None, sample_n=3):
    report = []
    for col in df.columns:
        null_pct = df[col].isna().mean() * 100
        unique_cnt = df[col].nunique(dropna=True)
        sample_vals = df[col].dropna().unique()[:sample_n]

        report.append({
            "column": col,
            "dtype": str(df[col].dtype),
            "null_%": round(null_pct, 2),
            "unique_count": unique_cnt,
            "sample_values": sample_vals
        })
    report_df = pd.DataFrame(report)

    if pk:
        dup_cnt = df.duplicated(subset=pk).sum()
        print(f"🔍 Duplicate check on {pk}: {dup_cnt} duplicate rows found")

    return report_df

# Chạy check cho từng bảng
dq_orders = data_quality_report(orders_parsed, pk=["order_id"])
dq_order_items = data_quality_report(order_items_parsed, pk=["order_id","product_id","variation_id"])
dq_pages = data_quality_report(pages_parsed, pk=["page_id"])
dq_warehouses = data_quality_report(warehouses_parsed, pk=["warehouse_id"])
dq_shipping = data_quality_report(shipping_addresses_parsed, pk=["order_id"])

# Hiển thị một số kết quả
print("\n=== FACT ORDERS ===")
display(dq_orders)

print("\n=== FACT ORDER ITEMS ===")
display(dq_order_items)

print("\n=== DIM PAGES ===")
display(dq_pages)

print("\n=== DIM WAREHOUSES ===")
display(dq_warehouses)

print("\n=== DIM SHIPPING ADDRESSES ===")
display(dq_shipping)


🔍 Duplicate check on ['order_id']: 0 duplicate rows found
🔍 Duplicate check on ['order_id', 'product_id', 'variation_id']: 625 duplicate rows found
🔍 Duplicate check on ['page_id']: 0 duplicate rows found
🔍 Duplicate check on ['warehouse_id']: 0 duplicate rows found
🔍 Duplicate check on ['order_id']: 0 duplicate rows found

=== FACT ORDERS ===


,column,dtype,null_%,unique_count,sample_values
0,order_id,object,0.00,40236,"[40616, 40615, 40614]"
1,shop_id,int64,0.00,1,[230361475]
2,page_id,object,0.02,73,"[377626778776391, 492088430643389, 43323114319..."
3,customer_id,object,0.00,36080,"[e906c5c7-ea19-42e5-a971-57cf09c94417, 6c5b40d..."
4,warehouse_id,object,0.00,24,"[5f7aaa7d-ae3c-48b6-8f0e-04207afae074, a0b8988..."
5,status,int64,0.00,7,"[0, 6, 5]"
6,status_name,object,0.00,7,"[new, canceled, returned]"
7,sub_status,float64,99.95,1,[1.0]
8,inserted_at,object,0.00,40236,"[2025-08-16T02:19:59.079610, 2025-08-15T12:56:..."
9,updated_at,object,0.00,40236,"[2025-08-16T02:19:59.339020, 2025-08-15T12:56:..."



=== FACT ORDER ITEMS ===


,column,dtype,null_%,unique_count,sample_values
0,order_id,object,0.0,34728,"[40458, 40455, 40454]"
1,product_id,object,0.0,85,"[358156d8-2777-4aab-b584-a26813fb4b95, ea5b416..."
2,variation_id,object,0.0,833,"[847cf093-8f90-4f01-85ae-800ec31432a1, 4e1af64..."
3,sku,object,100.0,0,[]
4,name,object,100.0,0,[]
5,quantity,int64,0.0,5,"[1, 0, 2]"
6,price,object,100.0,0,[]
7,discount,object,100.0,0,[]
8,total_price,object,100.0,0,[]
9,note,object,100.0,0,[]



=== DIM PAGES ===


,column,dtype,null_%,unique_count,sample_values
0,page_id,object,1.35,73,"[377626778776391, 492088430643389, 43323114319..."
1,page_name,object,1.35,72,"[Duy Mạnh Authentic, Sùn Outlet, Đăng Authentic]"
2,page_username,object,55.41,33,"[duymanhauth10, sunoutl, dangauth2006]"
3,account,object,1.35,73,"[377626778776391, 492088430643389, 43323114319..."
4,account_name,object,0.00,73,"[Duy Mạnh Authentic, Sùn Outlet, Đăng Authentic]"



=== DIM WAREHOUSES ===


,column,dtype,null_%,unique_count,sample_values
0,warehouse_id,object,0.00,24,"[5f7aaa7d-ae3c-48b6-8f0e-04207afae074, a0b8988..."
1,warehouse_name,object,0.00,3,"[Kho Winner, Kho mặc định, Kho hoàn]"
2,warehouse_address,object,91.67,2,"[Yên Xá, Xã Tân Triều, Huyện Thanh Trì, Hà Nội..."
3,province_id,object,91.67,1,[101]
4,district_id,object,91.67,2,"[10123, 10108]"
5,commune_id,object,91.67,2,"[1012335, 1010815]"
6,phone_number,object,91.67,1,[0344663171]



=== DIM SHIPPING ADDRESSES ===


,column,dtype,null_%,unique_count,sample_values
0,order_id,object,0.00,40236,"[40616, 40615, 40614]"
1,full_name,object,0.00,25327,"[Thinh Bui, Truong Minh, Quach Quach]"
2,phone_number,object,0.03,36100,"[0903693389, 0907809070, 0986533988]"
3,full_address,object,0.00,32550,"[, thôn Thái sơn, Xã Đình Chu, Huyện Lập Thạch..."
4,province_id,object,11.63,63,"[104, 101, 701]"
5,province_name,object,11.75,63,"[Vĩnh Phúc, Hà Nội, Hồ Chí Minh]"
6,district_id,object,11.66,701,"[10403, 10114, 70131]"
7,district_name,object,11.76,693,"[Huyện Lập Thạch, Quận Long Biên, Quận Phú Nhuận]"
8,commune_id,object,11.67,7343,"[1040371, 1011411, 7013103]"
9,commune_name,object,11.77,5614,"[Xã Đình Chu, Phường Thượng Thanh, Phường 2]"


### 6. Drop high-null columns (>70%) và ép kiểu dữ liệu

In [ ]:
def drop_high_nulls(df, threshold=0.7):
    cols_to_drop = [col for col in df.columns if df[col].isna().mean() > threshold]
    print(f"⚠️ Dropping {len(cols_to_drop)} columns (>{threshold*100}% null): {cols_to_drop}")
    return df.drop(columns=cols_to_drop)

orders_clean = drop_high_nulls(orders_parsed)
order_items_clean = drop_high_nulls(order_items_parsed)
pages_clean = drop_high_nulls(pages_parsed)
warehouses_clean = drop_high_nulls(warehouses_parsed)
shipping_clean = drop_high_nulls(shipping_addresses_parsed)


⚠️ Dropping 3 columns (>70.0% null): ['sub_status', 'time_assign_care', 'estimate_delivery_date']
⚠️ Dropping 6 columns (>70.0% null): ['sku', 'name', 'price', 'discount', 'total_price', 'note']
⚠️ Dropping 0 columns (>70.0% null): []
⚠️ Dropping 5 columns (>70.0% null): ['warehouse_address', 'province_id', 'district_id', 'commune_id', 'phone_number']
⚠️ Dropping 0 columns (>70.0% null): []


In [50]:
orders_clean.info()
print('Checktype')
order_items_clean.info()
print('Checktype')
pages_clean.info()
print('Checktype')
warehouses_clean.info()
print('Checktype')
shipping_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40236 entries, 0 to 40235
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   order_id                        40236 non-null  object 
 1   shop_id                         40236 non-null  int64  
 2   page_id                         40226 non-null  object 
 3   customer_id                     40236 non-null  object 
 4   warehouse_id                    40236 non-null  object 
 5   status                          40236 non-null  int64  
 6   status_name                     40236 non-null  object 
 7   inserted_at                     40236 non-null  object 
 8   updated_at                      40236 non-null  object 
 9   time_assign_seller              36574 non-null  object 
 10  total_quantity                  40236 non-null  int64  
 11  total_price                     40236 non-null  int64  
 12  total_discount                  

In [53]:
dtype_fact_orders = {
    "order_id": VARCHAR(100),           # object (id dạng chuỗi)
    "shop_id": BigInteger(),            # int64
    "page_id": VARCHAR(100),            # object
    "customer_id": VARCHAR(100),        # object
    "warehouse_id": VARCHAR(100),       # object

    "status": Integer(),                # int64
    "status_name": VARCHAR(50),         # object

    "inserted_at": DateTime(),          # object -> datetime
    "updated_at": DateTime(),
    "time_assign_seller": DateTime(),   # có null

    "total_quantity": Integer(),        # int64
    "total_price": BigInteger(),        # int64 (giá trị tiền lớn)
    "total_discount": BigInteger(),
    "total_price_after_sub_discount": BigInteger(),

    "cod": BigInteger(),
    "cash": BigInteger(),
    "transfer_money": BigInteger(),
    "charged_by_momo": BigInteger(),
    "charged_by_card": BigInteger(),
    "charged_by_qrpay": BigInteger(),
    "exchange_payment": BigInteger(),
    "prepaid": BigInteger(),

    "money_to_collect": BigInteger(),
    "shipping_fee": BigInteger(),
    "partner_fee": BigInteger(),
    "surcharge": Float(),               # float64
    "fee_marketplace": BigInteger(),
    "tax": BigInteger()
}


In [54]:
dtype_fact_order_items = {
    "order_id": VARCHAR(100),           # object
    "product_id": VARCHAR(100),         # object
    "variation_id": VARCHAR(100),       # object
    "quantity": Integer()               # int64
}


In [55]:
dtype_dim_pages = {
    "page_id": VARCHAR(100),
    "page_name": VARCHAR(255),
    "page_username": VARCHAR(255),
    "account": VARCHAR(100),
    "account_name": VARCHAR(255)
}


In [56]:
dtype_dim_warehouses = {
    "warehouse_id": VARCHAR(100),
    "warehouse_name": VARCHAR(255)
}


In [57]:
dtype_dim_shipping = {
    "order_id": VARCHAR(100),
    "full_name": VARCHAR(255),
    "phone_number": VARCHAR(50),
    "full_address": Text(),
    "province_id": VARCHAR(50),
    "province_name": VARCHAR(100),
    "district_id": VARCHAR(50),
    "district_name": VARCHAR(100),
    "commune_id": VARCHAR(50),
    "commune_name": VARCHAR(100)
}


In [58]:
# Ép kiểu datetime cho orders_clean
for col in ["inserted_at", "updated_at", "time_assign_seller"]:
    orders_clean[col] = pd.to_datetime(orders_clean[col], errors="coerce")


### 7. Load vào Silver

In [59]:
orders_clean.to_sql("fact_orders", silver_engine, if_exists="replace", index=False, dtype=dtype_fact_orders)
order_items_clean.to_sql("fact_order_items", silver_engine, if_exists="replace", index=False, dtype=dtype_fact_order_items)
pages_clean.to_sql("dim_pages", silver_engine, if_exists="replace", index=False, dtype=dtype_dim_pages)
warehouses_clean.to_sql("dim_warehouses", silver_engine, if_exists="replace", index=False, dtype=dtype_dim_warehouses)
shipping_clean.to_sql("dim_shipping_addresses", silver_engine, if_exists="replace", index=False, dtype=dtype_dim_shipping)

print("🚀 Loaded all cleaned & typed tables into Silver schema successfully!")


🚀 Loaded all cleaned & typed tables into Silver schema successfully!
